# Attachment Protocol Example - Alice

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/notebooks/attachment.ipynb)

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### Intialise a controller 

In [2]:
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8028
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)




### Confirm that an active connection exists

In [3]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'state': 'active', 'their_did': 'RgB2v726a7DzHWXfP9HXvD', 'initiator': 'external', 'routing_state': 'none', 'request_id': '1a095e43-8019-4d84-a59c-154abd4ccdb8', 'updated_at': '2020-09-20 14:01:49.946821Z', 'created_at': '2020-09-20 14:01:29.478241Z', 'connection_id': '80143a84-9fcb-4dfe-ac9e-aaf6f83df216', 'accept': 'manual', 'invitation_key': 'Ar7dsDPiQn58zhiKmwdwM1RfmQ5UHDkbHiKckd3GoSw', 'invitation_mode': 'once', 'their_label': 'Bob', 'my_did': 'MpprthBV8krDWReVcefM8t'}]
Connection : {'state': 'active', 'their_did': 'RgB2v726a7DzHWXfP9HXvD', 'initiator': 'external', 'routing_state': 'none', 'request_id': '1a095e43-8019-4d84-a59c-154abd4ccdb8', 'updated_at': '2020-09-20 14:01:49.946821Z', 'created_at': '2020-09-20 14:01:29.478241Z', 'connection_id': '80143a84-9fcb-4dfe-ac9e-aaf6f83df216', 'accept': 'manual', 'invitation_key': 'Ar7dsDPiQn58zhiKmwdwM1RfmQ5UHDkbHiKckd3GoSw', 'invitation_mode': 'once', 'their_label': 'Bob', 'my_did': 'MpprthBV8krDWReVcefM8t'}
Active Connect

### Instantiate Listener for the Attachment topic

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In [4]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)

attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(attach_controller.listen_webhooks())
attach_controller.register_listeners([attach_listener], defaults=True)

### Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [5]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': '7ff38303-2106-4556-a489-3c8f44aa932e'}


Here an image file is sent and is received by the handler in [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [6]:
import base64
from aiohttp import FormData

data = FormData()
data.add_field('file',
               open('openmined.jpg', 'rb'),
               filename='openmined.jpg',
               content_type='image/jpeg')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)

Attachment- Alice-> Bob
{'thread_id': '7a065c05-f90f-4b32-95b5-bb90d52c3a96'}



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial

In [8]:
response = await attach_controller.terminate()
print(response)


None
